**0. Python 支援不同DB**

In [0]:
import MySQLdb         # SQL用，但須使用Python2.7 
import sqlite3
import psycopg2
import pymysql
...

ModuleNotFoundError: ignored

**1. Connect to DB**

In [0]:
import sqlite3

# 建立資料庫連線，如果不存在，會先新增資料庫
conn = sqlite3.connect('test.sqlite')
# 建立 cursor 物件
cursor = conn.cursor()

# 若不存在，即建立一個資料表
sqlstr = 'CREATE TABLE IF NOT EXISTS table01\
         ("num" INTEGER PRIMARY KEY NOT NULL\
         ,"tel" TEXT)'

cursor.execute(sqlstr)

# 新增一筆紀錄
sqlstr = 'insert into table01 values(3,"02-1234567")'
cursor.execute(sqlstr)

# 主動更新資料庫
conn.commit()
# 關閉資料庫連線
conn.close()


**2. Fetchall**

In [0]:
import sqlite3
# 建立資料庫連線
conn = sqlite3.connect('test.sqlite')
cursor = conn.execute('select * from table01')
# return結果,抓取所有Query的row，回傳為一個list，其中元素為tuple
rows = cursor.fetchall()

print(rows)

for row in rows:
    print("{}\t{}".format(row[0],row[1]))

# 關閉資料庫
conn.close()

OperationalError: ignored

**3. Basic Operation**

In [0]:
import sqlite3

# 建立資料庫連線
conn = sqlite3.connect('test.sqlite')

def print_table01():
    cursor = conn.execute('select * from table01')
    rows = cursor.fetchall()
    for row in rows:
        print("{}\t{}".format(row[0],row[1]))
    print("====================================\n")

print("----- 原始資料表 -----")
print_table01()

print("----- 新增一筆紀錄 -----")
sqlstr = 'insert into table01 values(2, "04-7654321")'
conn.execute(sqlstr)
print_table01()

print("----- 變更一筆紀錄 -----")
sqlstr = 'update table01 set tel="02-7777777" where num=2'
conn.execute(sqlstr)
print_table01()

print("----- 刪除一筆紀錄 -----")
sqlstr = 'delete from table01 where num=2'
conn.execute(sqlstr)
print_table01()

# 資料庫更新
conn.commit()
# 關閉資料庫的連線
conn.close()

**4. 將資料抓進SQL**

In [1]:
import sqlite3,ast,hashlib,os,requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from google.colab import files

uploaded = files.upload()

# 如果連線錯誤
def requests_html(url):
	html = ""
	while html == "":
		try:
			html = requests.get(url).text.encode('utf-8')
		except:
			print("ˊ無法連線..")
			print('將在5分鐘後重啟')
			time.sleep(5)
			print('開始連線')
	return html

# 日期模板
format_pattern = "%Y-%m-%d %H:00"
# 現在時間
now = datetime.now()
#now = now + timedelta(hours=8) # 時區問題，須更新時區
now = now + timedelta(hours=7)
now = now.strftime(format_pattern)
# 印出更新時間(每小時)
print(now)

# 建立資料庫連線
conn = sqlite3.connect('DataBasePM25.sqlite')
#cur_path = os.path.dirname(__file__)
#print(cur_path)
#conn = sqlite3.connect(cur_path + '/' + 'DataBasePM25.sqlite')
# 建立cursor物件
cursor = conn.cursor()

# 建立一個資料表(如果不存在)
sqlstr = '''CREATE TABLE IF NOT EXISTS TablePM25PT ("SiteName" TEXT NOT NULL, "County" TEXT NOT NULL, "PM25" INTEGER, "DataCreationDate" TEXT,CONSTRAINT SN_DT UNIQUE(SiteName, DataCreationDate))'''
cursor.execute(sqlstr)

url = "http://opendata.epa.gov.tw/webapi/Data/ATM00625/?$skip=0&$top=1000&format=json"
# 讀取網頁原始碼
html = requests_html(url)

# 判斷網頁是否更新
new_md5 = hashlib.md5(html).hexdigest()
if os.path.exists('old_md5.txt'):
    with open('old_md5.txt','r') as f:
        old_md5 = f.read()
    with open('new_md5.txt','w') as f:
        f.write(new_md5)
else:
    with open('new_md5.txt','w') as f:
        f.write(new_md5)
    old_md5 = ""
        
if new_md5 != old_md5:
    print("*** 資料已更新...")
    sp = BeautifulSoup(html, 'html.parser')
    # 將網頁內容JSON資料轉換為 list, list中的元素是 dict
    jsondata = ast.literal_eval(sp.text)

    #　print(jsondata)

    # 刪除資料表內容
    # conn.execute("delete from TablePM25PT")
    # conn.commit()
    
    # 輸入每筆資料進table
    n = 1
    list1 = []
    for site in jsondata:
        # SiteName = site["SiteName"]
        SiteName = site["Site"]
        # PM25=0 if site["PM2.5"] == "" else int(site["PM25"])
        PM25 = 0 if site["PM25"] == "" else int(site["PM25"])
        County = site["county"]
        DataCreationDate = site["DataCreationDate"]
        
        if SiteName not in list1 and DataCreationDate == now:
            list1.append(SiteName)
            # 新增一筆紀錄
            sqlstr = "insert into TablePM25PT values('{}','{}',{},'{}')".format(SiteName,County,PM25,DataCreationDate)
            cursor.execute(sqlstr)
            n += 1
        conn.commit()
    b = True
else:
    print("*** 資料未更新...")
    b = False

# 如果資料更新，列印現在時間的每筆資料
if b == True:
  n = 1
  cursor = conn.execute("select * from TablePM25PT where DataCreationDate==\'%s\'" % now)
  rows = cursor.fetchall()
  for row in rows:
      #if row[3] == :
      #  print("目前時間不合，修正時間往前1小時")
      print("{} 站名:{}({})    PM2.5={}  Date={}".format(n,row[0],row[1],row[2],row[3]))
      n += 1
  print("================== 臺中市[%s] =====================" % now)
  cursor = conn.execute("select * from TablePM25PT where County == \"臺中市\" AND DataCreationDate == \'%s\'" % now)
  rows = cursor.fetchall()
  for row in rows:
    print("站名:{}    PM2.5={}".format(row[0],row[2]),end=" ")
    if row[2] < 36:
      print("*** 綠 ***")
    elif row[2] < 54:
      print("*** 黃 ***")
    elif row[2] < 71:
      print("*** 紅 ***")
    elif row[2] >= 71:
      print("*** 紫 ***")

# 下載檔案
files.download('DataBasePM25.sqlite')
files.download('old_md5.txt')
files.download('new_md5.txt')

conn.close()

2020-05-13 09:00
*** 資料已更新...
1 站名:富貴角(新北市)    PM2.5=33  Date=2020-05-13 09:00
2 站名:麥寮(雲林縣)    PM2.5=24  Date=2020-05-13 09:00
3 站名:關山(臺東縣)    PM2.5=19  Date=2020-05-13 09:00
4 站名:馬公(澎湖縣)    PM2.5=30  Date=2020-05-13 09:00
5 站名:金門(金門縣)    PM2.5=49  Date=2020-05-13 09:00
6 站名:馬祖(連江縣)    PM2.5=42  Date=2020-05-13 09:00
7 站名:埔里(南投縣)    PM2.5=15  Date=2020-05-13 09:00
8 站名:復興(高雄市)    PM2.5=38  Date=2020-05-13 09:00
9 站名:永和(新北市)    PM2.5=31  Date=2020-05-13 09:00
10 站名:竹山(南投縣)    PM2.5=21  Date=2020-05-13 09:00
11 站名:中壢(桃園市)    PM2.5=26  Date=2020-05-13 09:00
12 站名:三重(新北市)    PM2.5=55  Date=2020-05-13 09:00
13 站名:冬山(宜蘭縣)    PM2.5=23  Date=2020-05-13 09:00
14 站名:宜蘭(宜蘭縣)    PM2.5=23  Date=2020-05-13 09:00
15 站名:陽明(臺北市)    PM2.5=32  Date=2020-05-13 09:00
16 站名:花蓮(花蓮縣)    PM2.5=17  Date=2020-05-13 09:00
17 站名:臺東(臺東縣)    PM2.5=13  Date=2020-05-13 09:00
18 站名:恆春(屏東縣)    PM2.5=13  Date=2020-05-13 09:00
19 站名:潮州(屏東縣)    PM2.5=32  Date=2020-05-13 09:00
20 站名:屏東(屏東縣)    PM2.5=29  Date=2020-05-13 09:0

FileNotFoundError: ignored